
# Section III - Feature Extraction and Regressions

### Filter Methods

#### Part A -  Feature Extraction
The following feature extraction stragetices are common to performing data science. For this project, Principal Component Analysis was successful performed while Linear Discriminant Analysis and Kernal PCA required a more powerful CPU. Even though the project was run on a fairly power desktop with the GPU being utilized, these two feature extraction technique does not utilize the processing power of the GPU and therefore resulted in memory errors.   

* **Principal Component Analysis** — we get accustomed with Zillow's dataset and do basic data analysis: we collect basic statistics, plot a correlation matrix, compare train and test distributions.

* **Linear Discriminat Analysis** — we keep on getting memory errors when running a Linear Discriminant Analysis on the full dataset. The algortihm was reviously tested on a slice of the data which performed properly.

* **Kernal PCA** — we keep on getting memory errors when running a PCA on the full dataset. The algortihm was reviously tested on a slice of the data which performed properly.

#### Part B -  Multiple Linear Regression
The initial data set from zillow includes a separate file for the transactions,which includes the log error of actual sales price versus Zillow estimates, and a training data set that has 52 features.  



#### Part C - Support Vector Regression
The goal of the inital exploratory analysis is to assess if there is some strange oddities with the data. We will also track to see if there any interesting trends that we could help us determine which features are important. We may also need to do certain data transformation in order to produce meanful analysis.

#### Part D -  Decision Tree Regression
The initial data set from zillow includes a separate file for the transactions,which includes the log error of actual sales price versus Zillow estimates, and a training data set that has 52 features.  

#### Part E -  Random Forest Regressions
The initial data set from zillow includes a separate file for the transactions,which includes the log error of actual sales price versus Zillow estimates, and a training data set that has 52 features.  



In [38]:
import numpy as np   #Mathematics library
import matplotlib.pyplot as plt # for plotting
import pandas as pd  #manage datasets
import seaborn as sea

The FinishMissing.csv file is an output from the missing data appendix where every feature had a speicif strategy to deal with the missing values

In [39]:
# Dummy varies creaated, drop some features, Imputed missing data, 
df = pd.read_csv('FinishMissing.csv')
df=df.drop('Unnamed: 0',axis=1)
df.shape

(90811, 52)

In [40]:
#Drop outliers before splitting ex and y
avg = df['logerror'].mean()
std = df['logerror'].std()
upper_outlier = avg + 2*std
lower_outlier = avg - 2*std
print('upper', upper_outlier)
print(lower_outlier)
print(std)
print(avg)
#round up to drop outliers, til reasonable
df=df[ df.logerror > -0.32 ]
df=df[ df.logerror < 0.34 ]

upper 0.3373916901448827
-0.31575874699922946
0.16328760928602803
0.01081647157282662


In [41]:
df.shape

(88113, 52)

In [42]:
###############Create Dummy variables for Categorical data
df=pd.get_dummies(df,columns=['taxdelinquencyflag','fireplaceflag','propertyzoningdesc','propertycountylandusecode','hashottuborspa','airconditioningtypeid','architecturalstyletypeid','buildingqualitytypeid','buildingclasstypeid','decktypeid','fips','heatingorsystemtypeid','pooltypeid10','pooltypeid2','pooltypeid7','propertylandusetypeid','regionidcounty','regionidcity','regionidzip','regionidneighborhood','storytypeid','typeconstructiontypeid','month','day'],drop_first=True)

#Change dataframe name to help check track of major changes to dataset
dataset=df

In [43]:
#split into response and features
X = dataset.iloc[:, 2:].values
y = dataset.iloc[:,1].values


In [44]:
###################### Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [45]:
##################### Feature Scaling required for Neural Network & Feature Extraction
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [46]:
##### DIMENSIONALITY REDUCTION : PRINCIPAL COMPONENT ANALYSIS(PCA)
# Applying PCA * requires feature scaling
from sklearn.decomposition import PCA
pca = PCA(n_components = 10) # number of principal components explain variance, use '0' first
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
X = np.concatenate((X_train,X_test),axis=0)

## REGRESSION I: Fitting Multiple Linear Regression to the Training set


In [47]:
#############from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
# Predicting the Test set results
y_pred = regressor.predict(X_test)
y_pred.mean()

0.0068865756569827536

In [48]:
########### MEAN SQUARED ERROR

# Mean Square Error Train/Test Splt
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred, sample_weight=None, multioutput='uniform_average')
print(mse)
rmse = np.sqrt(mse)
print(rmse)


0.00580368263483
0.0761819049042


In [49]:
# 10-fold cross-validation with all three features
###For Feature Extraction Run codes below before cross validation

from sklearn.cross_validation import cross_val_score
scores = cross_val_score(regressor, X, y, cv=10, scoring='neg_mean_squared_error')
mse_scores = -scores
# calculate the average MSE
print(mse_scores.mean())
rmse_kfold = np.sqrt(mse_scores.mean())
print(rmse_kfold)

0.00580487541193
0.0761897329824


### REGRESSION II Fitting Support Vector Regression(SVR) to the dataset


In [50]:
##############from sklearn.svm import SVR
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

sc_y = StandardScaler()
y = sc_y.fit_transform(y)

regressor = SVR(kernel = 'rbf')
regressor.fit(X, y)


C:\Users\ceribrum\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\ceribrum\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [51]:
# Predicting a new result

y_pred = regressor.predict(X_test)
y_pred = sc_y.inverse_transform(y_pred)

In [52]:
# Mean Square Error Train/Test Splt
mse = mean_squared_error(y_test, y_pred, sample_weight=None, multioutput='uniform_average')
print(mse)
rmse = np.sqrt(mse)
print(rmse)

0.00600909524724
0.0775183542604


In [53]:
# 10-fold cross-validation with all three features
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(regressor, X, y, cv=10, scoring='neg_mean_squared_error')
mse_scores = -scores
# calculate the average MSE
print(mse_scores.mean())
rmse_kfold = np.sqrt(mse_scores.mean())
print(rmse_kfold)

1.01051584688
1.00524417277


## REGRESSION III: Fitting Decision Tree Regression to the dataset

In [54]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X, y)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best')

In [55]:
# Predicting a new result
y_pred = regressor.predict(X_test)
# Mean Square Error
mse = mean_squared_error(y_test, y_pred, sample_weight=None, multioutput='uniform_average')
print(mse)
rmse = np.sqrt(mse)
print(rmse)


1.35043223377
1.16208099277


In [56]:
# 10-fold cross-validation with all three features
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(regressor, X, y, cv=10, scoring='neg_mean_squared_error')
mse_scores = -scores
# calculate the average MSE
print(mse_scores.mean())
rmse_kfold = np.sqrt(mse_scores.mean())
print(rmse_kfold)

2.1722338982
1.47385002568


## REGRESSION IV:Fitting Random Forest Regression to the dataset

In [57]:
##############
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [58]:
# Predicting a new result
y_pred = regressor.predict(X_test)
# Mean Square Error
mse = mean_squared_error(y_test, y_pred, sample_weight=None, multioutput='uniform_average')
print(mse)
rmse = np.sqrt(mse)
print(rmse)

0.144197100071
0.379732932561


In [59]:
# 10-fold cross-validation with all three features
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(regressor, X, y, cv=10, scoring='neg_mean_squared_error')
mse_scores = -scores
# calculate the average MSE
print(mse_scores.mean())
rmse_kfold = np.sqrt(mse_scores.mean())
print(rmse_kfold)

1.16609565103
1.07985908851
